In [ ]:
%pip install git+https://GITHUB_TOKEN@github.com/phamson02/vnnews-graph-producer.git -q

In [ ]:
import json
from vnnews_graph_producer.scraper import async_get_last_24h_articles

articles = await async_get_last_24h_articles()

In [ ]:
from vnnews_graph_producer.data_models.entity import Entity
from vnnews_graph_producer.entity_extractor import get_entities_from_text

from tqdm import tqdm

entities: list[list[Entity]] = []
for article in tqdm(articles, desc="Extracting entities"):
    entities.append(get_entities_from_text(article.content))

In [2]:
from vnnews_graph_producer.data_models.graph import SubNewsGraph

subgraph = SubNewsGraph.full_graph_from_article(entities[0], articles[0])
for article, entities_in_article in zip(articles[1:], entities[1:]):
    subgraph += SubNewsGraph.full_graph_from_article(entities_in_article, article)


In [3]:
from vnnews_graph_producer.data_models.graph import NewsGraph

full_graph = NewsGraph.from_subgraph(subgraph, 2)

In [5]:
with open("dataset.json", "w") as f:
    f.write(json.dumps(full_graph.to_graph_data(), indent=4, ensure_ascii=False))

100%|██████████| 1000/1000 [00:00<00:00, 5634.84it/s]


BarnesHut Approximation  took  0.03  seconds
Repulsion forces  took  0.12  seconds
Gravitational forces  took  0.00  seconds
Attraction forces  took  0.00  seconds
AdjustSpeedAndApplyForces step  took  0.02  seconds


In [ ]:
!git clone https://GITHUB_TOKEN@github.com/phamson02/vnnews-graph.git

In [ ]:
%cd vnnews-graph

In [ ]:
!mv ../dataset.json public/

In [3]:
from datetime import datetime
from pytz import timezone

vn_tz = timezone("Asia/Ho_Chi_Minh")
now = datetime.now(vn_tz).strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
!git add public/dataset.json
!git config user.name github-actions && git config user.email github-actions@github.com
!git commit -m "Update dataset at {now} GMT+7"
!git push origin main